# [실습3] Langchain으로 문서 기반 챗봇 만들기 - PDF

## 실습 목표
---
[실습3] RAG를 위한 Vector Score, Retriever 에서 학습한 내용을 바탕으로 LangChain을 활용해서 입력된 문서를 요약해서 Context로 활용하는 챗봇을 개발합니다.

## 실습 목차
---

1. **교통 3대 혁신 전략 문서 벡터화:** RAG 챗봇에서 활용하기 위해 교통 3대 혁신 전략 파일을 읽어서 벡터화하는 과정을 실습합니다.

2. **RAG 체인 구성:** 이전 실습에서 구성한 미니 RAG 체인을 응용해서 간단한 교통 3대 혁신 전략 문서 기반 RAG 체인을 구성합니다.

3. **챗봇 구현 및 사용:** 구성한 RAG 체인을 활용해서 교통 3대 혁신 전략 문서 기반 챗봇을 구현하고 사용해봅니다.

## 실습 개요
---
RAG 체인을 활용해서 교통 3대 혁신 전략 문서 기반 챗봇을 구현하고 사용해봅니다.

## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

- OpenAI를 통해 gpt-4o-mini 모델을 불러옵니다.

## 1. 문서 벡터화
- RAG 챗봇에서 활용하기 위해 교통 3대 혁신 전략 파일을 읽어서 벡터화하는 과정을 실습합니다.

먼저, gpt-4o-mini 모델을 사용하는 ChatOpenAI 객체와 OpenAIEmbeddings 객체를 생성합니다.

In [2]:
import os


if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = ""


In [3]:
llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

C:\Users\timeo\AppData\Local\Temp\ipykernel_10348\3702560072.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini")


다음으로, PDF 문서를 불러와서 벡터화 해보겠습니다.
- 출처: 국토교통부의 교통 분야 3대 혁신 전략 보도자료
  - https://www.molit.go.kr/USR/NEWS/m_71/dtl.jsp?lcmspage=1&id=95089349
- 이 보고서는 2024년 교통 분야에서 3대 혁신 추진에 대해 다루는 보고서입니다. 
- 이를 활용해서 해당 정보를 빠르게 파악할 수 있도록 돕는 챗봇을 만들어야 하는 상황이라고 가정해 봅시다.

먼저, LangChain의 `PyPDFLoader`를 활용해서 교통 3대 혁신 전략 보고서의 텍스트를 추출하고, 페이지 별로 `Document`를 생성하여 저장합니다.

In [6]:
doc_path = "../data/교통_3대_혁신_전략.pdf"
loader = PyPDFLoader(doc_path)
docs = loader.load()

In [6]:
docs

[Document(metadata={'source': '교통_3대_혁신_전략.pdf', 'page': 0}, page_content='- 1 -\n부보도자료\n보도시점 : 2024. 1. 25.(목) 행사 진행시 별도 공지 / 배포 : 2024. 1. 24.(수)\n「교통 분야 3대 혁신 전략」\n출퇴근 30분 시대, 교통격차 해소에 134조원 투입\n- GTX-A 최초 개통, D·E·F 노선 임기 내 예타...윤석열 정부 ‘2기 GTX’ 본격화\n- 지방에도 광역급행철도 (x-TX) 도입 등 교통격차 해소\n- 2층 광역버스 늘리고 , 좌석 예약제 수도권 전역 확대\n- 연내 철도 지하화 선도사업 선정... 지하 고속도로 사업도 속도 높인다\n□ 정부는 1월 25일(목) 경기도 의정부시 에서 대통령 주재 ｢국민과 함께\n하는 민생 토론회 “출퇴근 30분 시대, 교통격차 해소”｣를 개최하고 \n｢교통 분야 3대 혁신 전략｣을 발표한다 .\n□ 이번 민생 토론회 에는 출퇴근 직장인 , 지자체 공무원  및 지역 주민, \n도시·교통 분야 전문가 등 각계각층의 국민이 참여하여 ‘국민이 바라는 \n교통상 ’에 대한 다양한 의견을 제시한다 .\n□ 정부는 국민의 목소리를 경청하고 , 출퇴근 30분 시대 실현과 교통격차 \n해소를 위해 다음 교통 분야 3대 혁신 전략을 추진해나갈 계획이다 .\n ❶ 속도 혁신 : 전국 GTX 시대를 통한 초연결 광역경제생활권 실현\n ㅇ GTX 사업 최초로 A노선 수서~동탄 구간을 ‘24.3월 개통하고 , 운정~\n서울역 구간을 연내 개통하는 등 ‘28년까지 순차 개통을 완료한다 .\n  - 또한, B·C노선도 연초부터 즉시 착공하고 적기 개통(B노선 ’30년, C노선 \n’28년)을 위해 공정을 차질없이 관리하는 등 1기 GTX 성과를 가시화\n하여 본격적으로 수도권 GTX 시대를 연다.'),
 Document(metadata={'source': '교통_3대_혁신_전략.pdf', 'page': 1}, page_content='- 2 -

생성된 Document의 수를 확인해봅시다.

In [7]:
print(len(docs))

29


다음으로, 각 Document의 길이를 확인해봅시다.

In [8]:
doc_len = [len(doc.page_content) for doc in docs]
print(doc_len)

[819, 1020, 835, 707, 122, 665, 821, 648, 452, 644, 584, 125, 863, 748, 590, 898, 688, 625, 1011, 805, 727, 704, 697, 753, 810, 719, 681, 839, 659]


1천자 미만의 문서가 대부분인걸 확인할 수 있습니다. 하지만 실제로는 이대로 그냥 사용할 경우, Context를 조절하는 것도 중요한 RAG의 요소입니다.
우선은 이대로 RAG 체인을 구성해 봅시다.

## 2. RAG 체인 구성
RAG 체인을 구성하기 위해 `Document`를 `OpenAIEmbeddings`를 활용해 벡터로 변환하고, FAISS DB를 활용하여 저장합니다.
- 변환 및 저장 과정은 약 3분 정도 소요됩니다.

In [9]:
vectorstore = FAISS.from_documents(
    docs,
    embedding=embeddings
)

In [10]:
db_retriever = vectorstore.as_retriever()

이전 실습에서 구성한 미니 RAG Chain과 비슷하게 Chain을 구성해 봅시다.
- 이번 챗봇의 역할은 교통 정보 제공을 위한 챗봇으로 고정했으므로, 역할을 별도로 인자로 전달할 필요가 없습니다.
- `RunnablePassthrough()`는 Chain의 이전 구성 요소에서 전달된 값을 그대로 전달하는 역할을 수행합니다.

### [TODO] pipe를 통해 체인을 구성해보세요.

In [11]:
def get_retrieved_text(docs):
    result = "\n".join([doc.page_content for doc in docs])
    return result

def init_chain():
    messages_with_contexts = [
        ("system", "당신은 교통 정보를 제공하기 위한 친절한 지원 챗봇입니다. 사용자가 입력하는 정보를 바탕으로 질문에 답하세요."),
        ("human", "정보: {context}.\n{question}."),
    ]

    prompt_with_context = ChatPromptTemplate.from_messages(messages_with_contexts)

    # 체인 구성
    # context에는 질문과 가장 비슷한 문서를 반환하는 db_retriever에 get_retrieved_text를 적용한 chain의 결과값이 전달됩니다.
    qa_chain = (
        {"context": db_retriever | get_retrieved_text, "question": RunnablePassthrough()}
        | prompt_with_context
        | llm
        | StrOutputParser()
    )
    
    return qa_chain

In [12]:
qa_chain = init_chain()

Chain 구성이 완료되었습니다.

## 3. 챗봇 구현 및 사용
- 구성한 RAG 체인을 활용해서 교통 3대 혁신 전략 문서 기반 챗봇을 구현하고 사용해봅니다.

방금 구현한 RAG Chain을 사용해서 교통 3대 혁신 전략 문서 기반 챗봇을 구현해볼 것입니다. 

그 전에, 별도로 RAG 기능을 추가하지 않은 LLM과 답변의 퀄리티를 비교해 봅시다.

### [TODO] 체인의 구성요소를 채워보세요.

In [13]:
messages_with_variables = [
    ("system", "당신은 교통 정보를 제공하기 위한 친절한 지원 챗봇입니다."),
    ("human", "{question}."),
]
prompt = ChatPromptTemplate.from_messages(messages_with_variables)
parser = StrOutputParser()
chain = prompt | llm | parser

In [14]:
print(chain.invoke("지하 고속도로 사업 정보에 대해 알려줘"))

지하 고속도로 사업은 도시 내 교통 혼잡을 해소하고, 도로 공간을 효율적으로 활용하기 위해 진행되는 프로젝트입니다. 일반적으로 이러한 사업은 다음과 같은 특징을 가지고 있습니다:

1. **목적**: 지하 고속도로는 차량 통행이 많은 지역에서 지상 도로의 혼잡을 줄이고, 대기오염 및 소음 문제를 개선하기 위해 설계됩니다. 

2. **구조**: 지하로 통과하는 고속도로는 여러 차선으로 구성될 수 있으며, 정체를 최소화하기 위한 설계가 이루어집니다. 

3. **설계 및 건설**: 지하 고속도로의 설계는 토질, 수자원, 기존 인프라와의 관계 등을 고려하여 신중하게 진행됩니다. 건설 과정에서는 굴착, 터널링 등 다양한 기술이 사용됩니다.

4. **비용 및 예산**: 지하 고속도로 사업은 일반 도로 건설보다 비용이 더 많이 들고, 긴 시간이 소요될 수 있습니다. 따라서 정부나 지방자치단체의 예산 계획이 중요합니다.

5. **환경 영향**: 지하 고속도로는 지상 공간을 활용하지 않기 때문에 도시의 녹지 공간을 유지할 수 있지만, 건설 과정에서 환경에 미치는 영향도 고려해야 합니다.

6. **운영 및 유지보수**: 고속도로가 개통된 후에는 지속적인 유지보수와 운영 관리가 필요합니다. 이를 위해 통행료 체계나 교통 관제 시스템이 도입될 수 있습니다.

각 도시나 지역에 따라 지하 고속도로 사업의 구체적인 내용이나 추진 상황은 다를 수 있으니, 특정 지역의 정보가 필요하시면 말씀해 주세요!


In [15]:
print(qa_chain.invoke("지하 고속도로 사업 정보에 대해 알려줘. 반드시 한글로 대답해"))

지하 고속도로 사업은 고속도로의 만성적인 교통 정체 문제를 해결하기 위해 추진되는 프로젝트입니다. 이 사업의 주요 내용은 다음과 같습니다:

1. **사업 착수 및 목표**:
   - 지하 고속도로를 통해 도로 용량을 입체적으로 확장하여 교통 정체 문제를 해소하는 것을 목표로 하고 있습니다.
   - 수도권 제 1순환도로(서창-김포)와 경부, 경인 지하 고속도로의 후속 절차를 신속히 진행하여 2026년부터 단계적으로 착공할 계획입니다.

2. **주요 사업 추진 일정**:
   - 수도권 제 1순환(서창-김포, 민자): 협상 및 실시협약 체결 후 2026년 착공 예정
   - 경부(용인-서울) 및 경인(인천-서울): 예비타당성 조사 중이며, 2027년 착공 예정
   - 지방 대도시권에서도 지역 교통 흐름을 고려한 지하도로 사업을 적극 발굴하고 있습니다. 예를 들어, 부산 사상-해운대 구간은 2028년 착공을 목표로 하고 있습니다.

3. **기술 개발 및 안전성**:
   - 안전한 지하도로를 위해 침수, 화재 등의 재난 및 사고 대응 기술 연구개발(R&D)을 2024년부터 2028년까지 진행할 계획입니다.
   - 운전자가 느끼는 불편(폐쇄감 등)을 개선하기 위한 다양한 방안도 검토하고 있습니다.

4. **상부 공간 활용**:
   - 지하도로 건설로 발생하는 상부 도로의 여유 공간을 공원, 녹지 등 친환경 공간으로 조성하거나 다양한 업무시설 등으로 활용하는 방안을 모색하고 있습니다.

이 프로젝트는 교통 혼잡 문제를 해결하고, 보다 안전하고 효율적인 교통 체계를 구축하기 위한 중요한 이니셔티브로, 수도권 및 지방 대도시권의 교통 개선에 기여할 것으로 기대됩니다. 추가적인 문의 사항이 있으시면 언제든지 질문해 주세요!


일반 체인은 아무런 출처가 없는 답변을 생성한 반면, RAG 기능을 추가한 챗봇은 데이터를 기반으로 상대적으로 정확한 답변을 하는 것을 확인할 수 있습니다. 

이제 챗봇을 한번 사용해 봅시다.

In [16]:
qa_chain = init_chain()
while True:
    question = input("질문을 입력해주세요 (종료를 원하시면 '종료'를 입력해주세요.): ")
    if question == "종료":
        break
    else:
        result = qa_chain.invoke(question)
        print(result)

지하 고속도로 사업은 고속도로의 만성적인 교통 정체 문제를 해결하기 위해 추진되고 있는 프로젝트입니다. 주요 내용은 다음과 같습니다:

1. **사업 개요**:
   - 지하 고속도로를 통해 도로 용량을 입체적으로 확장하려고 하며, 수도권과 지방 대도시권을 대상으로 하고 있습니다.

2. **수도권 사업**:
   - 수도권 제 1순환 고속도로 (서창-김포), 경부 고속도로 (용인-서울), 경인 고속도로 (인천-서울) 등의 후속 절차를 신속히 진행하여 2026년부터 단계적으로 착공할 예정입니다.
   - 각 고속도로의 사업 절차는 예비 타당성 평가와 설계를 포함하여 진행됩니다.

3. **지방 사업**:
   - 부산 사상-해운대 구간과 같은 지방 대도시권의 지하도로 사업도 발굴하여 추진할 계획입니다. 부산 구간은 2024~2025년 협상 및 실시협약 체결 후 2028년 착공 예정입니다.

4. **기술 개발 및 안전성**:
   - 지하도로의 안전성을 확보하기 위해 재난 및 사고 대응 기술 연구개발(R&D)이 진행될 예정입니다 (2024년 4월~2028년 12월).

5. **상부 공간 활용**:
   - 지하도로 건설로 발생할 수 있는 상부 도로 여유 공간을 공원, 녹지 등 친환경 공간으로 조성하거나 다양한 용도로 활용하는 방안도 검토 중입니다.

6. **사업비 및 관리**:
   - 이번 사업은 약 11조 원의 광역교통개선대책 사업비를 활용하여 집중 투자할 계획이며, 사업 진행 속도를 높이기 위해 절차 개선도 이루어질 예정입니다.

이와 같은 지하 고속도로 사업은 교통 혼잡을 해소하고, 지역 교통 흐름을 개선하기 위한 중요한 프로젝트입니다. 추가적인 질문이 있으시면 언제든지 말씀해 주세요!
안녕하세요! 교통 정보에 대해 궁금한 점이 있으신가요? 제공된 정보는 교통 분야의 혁신과 관련된 내용이 많습니다. 특정 내용이나 궁금한 점에 대해 질문해 주시면 최선을 다해 도움을 드리겠습니다!


저희는 이전 챕터에서 구현한 챗봇이 가지고 있는 문제점 중 '문서나 데이터 기반 추론이 불가능하다.'를 완화했습니다.